### puzzle 1 ###

In [1]:
test_map = '''..#
#..
...'''.split('\n')
test_map

['..#', '#..', '...']

The initial map is ```nrows``` rows of characters, with each row made of ```ncols``` columns. That puts the row,col origin at the upper left, but I really want an x/y coordinate system with the origin in the center of the initial map. The results is:

```python
x(row, col) = -(ncols//2) + col
y(row, col) = +(nrows//2) - row
```

In [2]:
def initial_infected(map):
    nrows = len(map)
    ncols = len(map[0])
    locations = [(-(ncols//2) + col, (nrows//2) - row) for row in range(nrows) 
                                                   for col in range(ncols) if map[row][col] == '#']
    return locations

In [3]:
len(test_map)//2, len(test_map[0])//2

(1, 1)

In [4]:
test_initial_infected = initial_infected(test_map)
test_initial_infected

[(1, 1), (-1, 0)]

In [5]:
puzzle_map = open('day22_input').read().split('\n')[:-1]
puzzle_initial_infected = initial_infected(puzzle_map)
len(puzzle_map), len(puzzle_map[0])

(25, 25)

The direction that the "virus" is facing is N, S, E, or W. For my purposes, I'm going to define an angular coordinate system so that 0 degrees corresponds to E, 90 degrees to N, 180 degrees to E, and 270 degrees to S. That way turning right or left from the current direction corresponds to adding or subtracting 90 degrees to the current angle (modulo 360). Advancing one step in that coordinate system corresponds to:

$$ x_\text{new} = x_\text{old} + \cos(\theta) $$
and
$$ y_\text{new} = y_\text{old} + \sin(\theta). $$

Note that for only four angles, we can just pre-compute the cosine and sine values:

In [6]:
cs = {0: 1, 90: 0, 180: -1, 270: 0}
sn = {0: 0, 90: 1, 180: 0,  270: -1}

In [7]:
def burst(infected, pos, ang):
    if pos in infected:
        infected.remove(pos)
        newang = (ang - 90) % 360
        x,y = pos
        newpos = (x+cs[newang], y+sn[newang])
        return infected, newpos, newang
    else:
        infected.append(pos)
        newang = (ang + 90) % 360
        x,y = pos
        newpos = (x+cs[newang], y+sn[newang])
        return infected, newpos, newang      

In [8]:
infected = test_initial_infected.copy()
pos = (0, 0)
ang = 90
for i in range(7):
    print(pos, ang, infected, '--> ', end='')
    infected, pos, ang = burst(infected, pos, ang)
    print(pos, ang, infected)
print(infected)

(0, 0) 90 [(1, 1), (-1, 0)] --> (-1, 0) 180 [(1, 1), (-1, 0), (0, 0)]
(-1, 0) 180 [(1, 1), (-1, 0), (0, 0)] --> (-1, 1) 90 [(1, 1), (0, 0)]
(-1, 1) 90 [(1, 1), (0, 0)] --> (-2, 1) 180 [(1, 1), (0, 0), (-1, 1)]
(-2, 1) 180 [(1, 1), (0, 0), (-1, 1)] --> (-2, 0) 270 [(1, 1), (0, 0), (-1, 1), (-2, 1)]
(-2, 0) 270 [(1, 1), (0, 0), (-1, 1), (-2, 1)] --> (-1, 0) 0 [(1, 1), (0, 0), (-1, 1), (-2, 1), (-2, 0)]
(-1, 0) 0 [(1, 1), (0, 0), (-1, 1), (-2, 1), (-2, 0)] --> (-1, 1) 90 [(1, 1), (0, 0), (-1, 1), (-2, 1), (-2, 0), (-1, 0)]
(-1, 1) 90 [(1, 1), (0, 0), (-1, 1), (-2, 1), (-2, 0), (-1, 0)] --> (0, 1) 0 [(1, 1), (0, 0), (-2, 1), (-2, 0), (-1, 0)]
[(1, 1), (0, 0), (-2, 1), (-2, 0), (-1, 0)]


In [9]:
def activity(map, bursts):
    infected = initial_infected(map)
    num_added_infections = 0
    pos = (0, 0)
    ang = 90
    for i in range(bursts):
        curr_num = len(infected)
        infected, pos, ang = burst(infected, pos, ang)
        if len(infected) > curr_num:
            num_added_infections += 1
    return num_added_infections

In [10]:
assert(41 == activity(test_map, 70))
assert(5587 == activity(test_map, 10000))

In [11]:
activity(puzzle_map, 10000)

5305

### puzzle 2 ###

More complicated rules, but same basic idea. Start at the center facing up. Change the current location, and move one step forward in a particular direction. Instead of infected and clean, though, there's now "weakened" and "flagged" states. My first effort had separate infected, flagged, and weakened structures, but moving a position from one list to the next was way too slow.

In [12]:
def initial_infected2(map):
    nrows = len(map)
    ncols = len(map[0])
    nodes = {}
    for row in range(nrows):
        for col in range(ncols):
            pos = (-(ncols//2) + col, (nrows//2) - row)
            if map[row][col] == '#':
                nodes[pos] = 'I'
            elif map[row][col] == '.':
                nodes[pos] = 'C'
            else:
                raise ValueError('Bad character in the map: {}'.format(map[row][col]))
    return nodes

In [13]:
initial_infected2(test_map)

{(-1, -1): 'C',
 (-1, 0): 'I',
 (-1, 1): 'C',
 (0, -1): 'C',
 (0, 0): 'C',
 (0, 1): 'C',
 (1, -1): 'C',
 (1, 0): 'C',
 (1, 1): 'I'}

In [14]:
deltheta = {'C': 90, 'W': 0, 'I': -90, 'F': 180}
next_status = {'C': 'W', 'W': 'I', 'I': 'F', 'F': 'C'}
def burst2(nodes, pos, ang):
    if pos not in nodes:
        nodes[pos] = 'C'
    cond = nodes[pos]
    newang = (ang + deltheta[cond]) % 360
    x,y = pos
    newpos = (x + cs[newang], y + sn[newang])
    nodes[pos] = next_status[cond]
    if nodes[pos] == 'I':
        infection_added = True
    else:
        infection_added = False
    return nodes, newpos, newang, infection_added   

In [15]:
def activity2(map, bursts):
    nodes = initial_infected2(map)
    pos = (0, 0)
    ang = 90
    num_added_infections = 0
    for i in range(bursts):
        nodes, pos, ang, infection_added = burst2(nodes, pos, ang)
        if infection_added:
            num_added_infections += 1
    return num_added_infections

In [16]:
assert(26 == activity2(test_map, 100))
assert(2511944 == activity2(test_map, 10000000))

In [17]:
activity2(puzzle_map, 10_000_000)

2511424